###  Libraries

In [1]:
from KernelKMeans import KernelKMeans
from GlobalKernelKMeans import GlobalKernelKMeans, GlobalKernelKMeansPP 
from Datasets.Datasets_Functions import *
from Common_Modules.Graph import Graph
from Common_Modules.General_Functions import General_Functions

### Switches

In [2]:
nc_is_enabled = True
kernelKMeans_is_enabled = True
globalKernelKMeans_is_enabled = True
globalKernelKMeansPP_is_enabled = True

### Global Variables

In [3]:
K = 10
n_candidates = n_init = 50

### Graph

In [4]:
dataset_names = ["EMAIL-EU-CORE-TD1", "EMAIL-EU-CORE", "CA-GRQC"]
dataset_name = dataset_names[0]

graph = Graph()
graph.create_G_from_file(f"Graphs\\{dataset_name}.txt")
if(nc_is_enabled):
    kernel_matrix = graph.create_kernel_matrix_from_adj_matrix(D = graph.degree_matrix)
else:
    kernel_matrix = graph.create_kernel_matrix_from_adj_matrix()

data_dir_path = f"Results\\{dataset_name}\\{n_init}\\"
General_Functions().create_directory(data_dir_path)

Directory 'Results\EMAIL-EU-CORE-TD1\50\' already exists.


### Kernel kMeans

In [5]:
if(kernelKMeans_is_enabled):
    initialization_methods = ["forgy", "random", "k-means++"]
    init = initialization_methods[2]
    kernelKMeans = KernelKMeans(n_clusters=K, kernel_matrix=kernel_matrix, n_init=n_init, init=init, verbose=0)
    if(nc_is_enabled):
        kernelKMeans.fit(sample_weights = graph.sample_weights)
    else:
        kernelKMeans.fit()
        
    n_iters_ = sum(kernelKMeans.n_iters_.values())
    execution_time_ = sum(kernelKMeans.execution_times_.values())
    inertia_ = kernelKMeans.inertia_
    print(f"K: {K}, MSE: {inertia_}, ITERATIONS: {n_iters_}, EXECUTION TIME: {execution_time_}")

K: 10, MSE: 476.1686720705982, ITERATIONS: 53, EXECUTION TIME: 1.1272978782653809


### Global Kernel kMeans

In [6]:
if(globalKernelKMeans_is_enabled):
    globalKernelKMeans = GlobalKernelKMeans(n_clusters=K, kernel_matrix=kernel_matrix, data_dir_path=f"{data_dir_path}GlobalKernelKMeans_Dataset_{dataset_name}_K_{K}_Inits_{n_init}_Results.csv", verbose=1)
    
    if(nc_is_enabled):
        globalKernelKMeans.fit(sample_weights = graph.sample_weights)
    else:
        globalKernelKMeans.fit()

Solving Kernel 2-means
Solved 2-means MSE: 489.34311560757675 in 0.7289865016937256s
Solving Kernel 3-means
Solved 3-means MSE: 487.07696707468915 in 0.6972260475158691s
Solving Kernel 4-means
Solved 4-means MSE: 484.90631792135946 in 0.762845516204834s
Solving Kernel 5-means
Solved 5-means MSE: 482.76216821756486 in 0.8544981479644775s
Solving Kernel 6-means
Solved 6-means MSE: 480.6621711968496 in 0.9233641624450684s
Solving Kernel 7-means
Solved 7-means MSE: 478.6624356179101 in 0.9641482830047607s
Solving Kernel 8-means
Solved 8-means MSE: 476.7780328751079 in 0.8835182189941406s
Solving Kernel 9-means
Solved 9-means MSE: 474.95765357388393 in 0.8984534740447998s
Solving Kernel 10-means
Solved 10-means MSE: 473.1597698015021 in 0.8758292198181152s
Total execution time was 7.588869571685791s


In [7]:
if(globalKernelKMeansPP_is_enabled):
    sampling_methods = ["sequential", "batch"]
    sampling = sampling_methods[1]
    globalKernelKMeansPP = GlobalKernelKMeansPP(n_clusters=K, kernel_matrix=kernel_matrix, n_candidates=n_candidates, sampling=sampling, data_dir_path=f"{data_dir_path}GlobalKernelKMeans_Dataset_{dataset_name}_Sampling_{sampling}_K_{K}_Inits_{n_init}_Results.csv", verbose=1)
    
    if(nc_is_enabled):
        globalKernelKMeansPP.fit(sample_weights = graph.sample_weights)
    else:
        globalKernelKMeansPP.fit()    

Solving 2-means
Solved 2-means MSE: 490.2002580645161 in 0.11755537986755371s
Solving 3-means
Solved 3-means MSE: 487.7433742059389 in 0.11359524726867676s
Solving 4-means
Solved 4-means MSE: 486.1436329380472 in 0.1128087043762207s
Solving 5-means
Solved 5-means MSE: 484.5438918040056 in 0.11662864685058594s
Solving 6-means
Solved 6-means MSE: 482.944150803918 in 0.11075806617736816s
Solving 7-means
Solved 7-means MSE: 480.77350218469974 in 0.11134791374206543s
Solving 8-means
Solved 8-means MSE: 478.6740232782267 in 0.11939287185668945s
Solving 9-means
Solved 9-means MSE: 477.07402462542655 in 0.14401030540466309s
Solving 10-means
Solved 10-means MSE: 475.27611363129324 in 0.12019586563110352s
Total execution time was 1.0662930011749268s
